In [1]:
## Module Import 

from __future__ import print_function
from torch import nn, optim, cuda
from torch.utils import data
from torchvision import datasets, transforms
import torch.nn.functional as F
import time
import matplotlib.pyplot as plt


## Google Drive Mount 

from google.colab import drive
drive.mount('/content/hw_drive')


## Trainig Setup

batch_size = 64
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'Training MNIST Model on {device}\n{"="*44}')


## MNIST dataset Download 
train_dataset = datasets.MNIST(root='/content/hw_drive/My Drive/mai1/mnist_data/', 
                              train=True, 
                              transform=transforms.ToTensor(),
                              download=True)
test_dataset = datasets.MNIST(root='/content/hw_drive/My Drive/mai1/mnist_data/',
                             train=False,
                             transform=transforms.ToTensor())

## Data loader
train_loader = data.DataLoader(dataset=train_dataset,
                               batch_size=batch_size,
                               shuffle=True)
test_loader = data.DataLoader(dataset=test_dataset,
                              batch_size=batch_size,
                              shuffle=False)


################## CNN

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.l1 = nn.Conv2d(1, 16, 3,padding=1)
    self.l2 = nn.Conv2d(16,16, 3,padding=1)
    self.l3 = nn.Conv2d(16,32, 3,padding=1)
    self.l4 = nn.Linear(1568, 120)
    self.l5 = nn.Linear(120, 10)

# 그레이 이미지 : n x 1 x  w x h
# 컬러 이미지 : n x 3 x w x h
  def forward(self, x):  
    # Mnist input : 28(w)x28(h)x1(c)  
    x = F.max_pool2d(F.relu(self.l1(x)), 2) 
    x = F.relu(self.l2(x))
    x = F.max_pool2d(F.relu(self.l3(x)), 2)      
    x = x.view(-1, 1568)
    x = F.relu(self.l4(x))
    return self.l5(x)

# ################## 연습


# class Net(nn.Module):
#   def __init__(self):
#     super(Net, self).__init__()
#     self.l1 = nn.Conv2d(3, 24, 3, padding=1)
#     self.l2 = nn.Conv2d(24,48, 5, padding=1)
#     self.l3 = nn.Linear(3888, 1000)
#     self.l4 = nn.Linear(1000, 5)

# # 그레이 이미지 : n x 1 x  w x h
# # 컬러 이미지 : n x 3 x w x h
#   def forward(self, x):  
#     # Mnist input : 36(w)x36(h)x3(c)  
#     x = F.max_pool2d(F.relu(self.l1(x)), 2)
#                     # 36x36x24 > 18x18x24 (code: 24,18,18) 
#     x = F.max_pool2d(F.relu(self.l3(x)), 2)      
#                     # 18x18x48 > 9x9x48
#     print(x.shape) # (n, 48, 9, 9)
#     x = x.view(-1, 81*48)
#     x = F.relu(self.l3(x))
#     return self.l4(x)



## Network Architecture Definition

# class Net(nn.Module):
#   def __init__(self):
#     super(Net, self).__init__()
#     self.l1 = nn.Linear(784, 520)
#     self.l2 = nn.Linear(520, 320)
#     self.l3 = nn.Linear(320, 240)
#     self.l4 = nn.Linear(240, 120)
#     self.l5 = nn.Linear(120, 10)

#   def forward(self, x):
#     x = x.view(-1, 784)   #Flatten the data ( n, 1, 28, 28 ) -> (n, 784)
#     x = F.relu(self.l1(x))
#     x = F.relu(self.l2(x))
#     x = F.relu(self.l3(x))
#     x = F.relu(self.l4(x))
#     return self.l5(x)

## Network Load

model = Net()
model.to(device)


## Training Setup

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


## Train Function 

def train(epoch):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % 10 == 0:
      print('Train Epoch : {} | Batch Status : {}/{} ({:.0f}%) | Loss : {:.6f}'.format(
          epoch, batch_idx*len(data), len(train_loader.dataset),
          100. * batch_idx / len(train_loader), loss.item()))


## Test Function

def test():
  model.eval()
  test_loss = 0
  correct = 0
  for data, target in test_loader:
    data, target = data.to(device), target.to(device)
    output = model(data)

    # sum up batch loss
    test_loss += criterion(output, target).item()

    # get the index of the max
    pred = output.data.max(1, keepdim=True)[1]
    correct += pred.eq(target.data.view_as(pred)).cpu().sum()

  test_loss /= len(test_loader.dataset)
  print(f'==================\nTest set: Average loss : {test_loss:.4f}, Accuracy : {correct}/{len(test_loader.dataset)}'
        f'({100. * correct / len(test_loader.dataset):.0f}%)')
  

## Main

if __name__ == '__main__':
 
  since = time.time()
  for epoch in range(1, 10):
    epoch_start = time.time()
    train(epoch)
    m, s = divmod(time.time() - epoch_start, 60)
    print(f'Training time: {m:.0f}m {s:.0f}s')
    
    test()
    m, s = divmod(time.time() - epoch_start, 60)
    print(f'Tesing time: {m:.0f}m {s:.0f}s')

  m, s = divmod(time.time() - epoch_start, 60)
  print(f'Total time : {m:.0f}m {s: .0f}s \nModel was trained on {device}!')

Mounted at /content/hw_drive
Training MNIST Model on cuda
Train Epoch : 1 | Batch Status : 0/60000 (0%) | Loss : 2.297383
Train Epoch : 1 | Batch Status : 640/60000 (1%) | Loss : 2.306450
Train Epoch : 1 | Batch Status : 1280/60000 (2%) | Loss : 2.296822
Train Epoch : 1 | Batch Status : 1920/60000 (3%) | Loss : 2.299966
Train Epoch : 1 | Batch Status : 2560/60000 (4%) | Loss : 2.292562
Train Epoch : 1 | Batch Status : 3200/60000 (5%) | Loss : 2.291406
Train Epoch : 1 | Batch Status : 3840/60000 (6%) | Loss : 2.300788
Train Epoch : 1 | Batch Status : 4480/60000 (7%) | Loss : 2.298200
Train Epoch : 1 | Batch Status : 5120/60000 (9%) | Loss : 2.300053
Train Epoch : 1 | Batch Status : 5760/60000 (10%) | Loss : 2.297195
Train Epoch : 1 | Batch Status : 6400/60000 (11%) | Loss : 2.289460
Train Epoch : 1 | Batch Status : 7040/60000 (12%) | Loss : 2.299378
Train Epoch : 1 | Batch Status : 7680/60000 (13%) | Loss : 2.291673
Train Epoch : 1 | Batch Status : 8320/60000 (14%) | Loss : 2.287307
Tra